# ETL Financeiro - Pipeline de Carteira de Ativos

Uma gestora de investimentos monitora uma carteira de ativos financeiros de forma que possa dar o melhor suporte possível para um determinado cliente. Esse cliente possui em sua carteira pessoal ações, criptomoedas, metais preciosos, e outras moedas estrangeiras. A gestora achou interessante concentrar todas as informações necessarias de maneira organizada, padronizada e de facil acesso por meio de um aplicativo web do qual possa ser acessado diretamente de qualquer navegador. https://carteira-ativos-app-wcs3v2ktsowjkgf7s47oms.streamlit.app/

Em virtude da necessidade da gestora financeira, esse projeto tem como objetivo criar um pipeline de dados por meio de ETL (Extract, Transform, Load), pois os dados foram retirados de fontes distintas, dos quais precisaram ser transformados (limpos, convertidos e padronizados), de forma que fossem carregados em um destino único. Assim a aplicação web criada se torna uma base confiavel para a empresa e para o cliente com dados corretos, atualizados e unificados de forma que economize o tempo de ambos pela prticidade. 

Para o desenvolvimento desse projeto foram utilizados:

1. Selenium para o acesso automatico ao site https://www.infomoney.com.br/cotacoes/b3/acao/banco-do-brasil-bbas3/historico/
2. Web screping para a coleta dos dados das cotações das ações do Banco do Brasil do site acessado.
3. API's (yfinancial, https://min-api.cryptocompare.com/data/v2/histoday, https://www.alphavantage.co/query
4. Biblioteca Pandas para a limpeza e organização dos dados.
4. Streamlit para a criação da aplicação web, de forma que as tabelas fiquem de maneira facil e acessível.

AS seguintes etapas foram realizadas:

1. Instalação e importação das bibliotecas
2. Utilização de selenium e webscraping para a obetenção do historico de cotações da primeira ação
3. Utilização das API's em cada obtenção do historico dos ativos
4. Limpeza dos dados em cada obtenção (Remoção, criação, renomeação e reorganização das colunas)
5. Utilização do Streamlit para a criação da aplicação web.

In [1]:
!pip install yfinance
!pip install pandas_datareader

In [16]:
import pandas as pd
import yfinance as yf
import datetime
import requests

In [ ]:
hoje = datetime.datetime.today().strftime('%Y-%m-%d')

In [18]:

bbdc3 = yf.Ticker("BBDC3.SA")
df_bbdc3 = bbdc3.history(start="2010-01-01", end=hoje).reset_index()

df_bbdc3 = df_bbdc3.drop(columns=['Dividends', 'Stock Splits'])
df_bbdc3.columns = ["Data", "Abertura", "Máxima", "Mínima", "Fechamento", "Volume"]
df_bbdc3['Variação (%)'] = df_bbdc3['Fechamento'].pct_change().mul(100).round(2)

df_bbdc3['Data'] = pd.to_datetime(df_bbdc3['Data'])
df_bbdc3 = df_bbdc3.sort_values('Data', ascending=False).reset_index(drop=True)
df_bbdc3['Data'] = df_bbdc3['Data'].dt.strftime('%d/%m/%Y')
df_bbdc3.head(5)

,Data,Abertura,Máxima,Mínima,Fechamento,Volume,Variação (%)
0,27/06/2025,14.15,14.31,14.11,14.28,6460300,-0.21
1,26/06/2025,14.21,14.32,14.02,14.31,7429000,0.85
2,25/06/2025,14.21,14.33,14.10,14.19,9468600,-0.49
3,24/06/2025,14.26,14.42,14.15,14.26,8276900,0.49
4,23/06/2025,14.25,14.25,13.98,14.19,8170600,-0.70


In [19]:
ouro = yf.Ticker("GC=F")

df_ouro = ouro.history(start="2010-01-01", end=hoje).reset_index()
df_ouro = df_ouro.drop(columns=['Dividends','Stock Splits'])
df_ouro.columns = ["Data","Abertura","Máxima","Mínima","Fechamento","Volume"]
df_ouro['Variação'] = df_ouro['Fechamento'].pct_change().mul(100).round(2)
df_ouro = df_ouro[::-1].reset_index(drop=True)
df_ouro['Data'] = df_ouro['Data'].dt.strftime('%d/%m/%Y')
df_ouro.head(5)


,Data,Abertura,Máxima,Mínima,Fechamento,Volume,Variação
0,27/06/2025,3341.300049,3341.399902,3266.500000,3286.100098,213305,-1.42
1,26/06/2025,3332.399902,3333.500000,3332.399902,3333.500000,2058,0.19
2,25/06/2025,3321.600098,3331.199951,3310.600098,3327.100098,373,0.29
3,24/06/2025,3358.000000,3358.000000,3301.000000,3317.399902,538,-1.79
4,23/06/2025,3365.899902,3387.899902,3350.000000,3377.699951,30,0.29


In [20]:
prata = yf.Ticker("SI=F")

df_prata = prata.history(start="2010-01-01", end=hoje).reset_index()
df_prata = df_prata.drop(columns=['Dividends','Stock Splits'])
df_prata.columns = ["Data","Abertura","Máxima","Mínima","Fechamento","Volume"]
df_prata['Variação'] = df_prata['Fechamento'].pct_change().mul(100).round(2)
df_prata = df_prata[::-1].reset_index(drop=True)
df_prata['Data'] = df_prata['Data'].dt.strftime('%d/%m/%Y')
df_prata.head(5)

,Data,Abertura,Máxima,Mínima,Fechamento,Volume,Variação
0,27/06/2025,36.889999,36.974998,35.950001,36.165001,65468,-1.15
1,26/06/2025,36.310001,36.585999,36.310001,36.585999,43116,1.39
2,25/06/2025,36.084999,36.084999,36.084999,36.084999,2,1.08
3,24/06/2025,35.830002,35.830002,35.599998,35.701000,10,-1.25
4,23/06/2025,36.160000,36.165001,36.153000,36.153000,7,0.49


In [11]:
#Bitcoin cryptocompare

url = 'https://min-api.cryptocompare.com/data/v2/histoday'
api = '37acf552f80fe45daae9c8cb8904d98e0ee288704e6e16a0def95dbb850d4687'
params = {
    'fsym': 'BTC',               
    'tsym': 'BRL',              
    'limit': 2000,              
    'api_key': api  
}

response = requests.get(url, params=params)
dados = response.json()

df_bitcoin = pd.DataFrame(dados['Data']['Data'])
df_bitcoin = df_bitcoin.drop(columns=['volumeto','conversionType', 'conversionSymbol'])
df_bitcoin = df_bitcoin[['time','open','high','low','close','volumefrom']]
df_bitcoin.columns = ['Data','Abertura','Máxima','Mínima','Fechamento','Volume']
df_bitcoin['Variação'] = df_bitcoin['Fechamento'].pct_change().mul(100).round(2)
df_bitcoin['Data'] = pd.to_datetime(df_bitcoin['Data'], unit='s').dt.strftime('%d/%m/%Y')
df_bitcoin = df_bitcoin[::-1]


df_bitcoin.head(5)

,Data,Abertura,Máxima,Mínima,Fechamento,Volume,Variação
2000,28/06/2025,591112.95,594391.40,589956.25,592838.05,54.31,0.29
1999,27/06/2025,590086.05,594529.91,585080.64,591112.95,130.80,0.17
1998,26/06/2025,599440.09,603196.54,588778.39,590086.05,174.20,-1.56
1997,25/06/2025,585928.77,601809.64,584833.26,599440.09,171.03,2.31
1996,24/06/2025,580828.37,587476.77,574851.55,585928.77,180.59,0.88


In [7]:
url = 'https://min-api.cryptocompare.com/data/v2/histoday'
api = '37acf552f80fe45daae9c8cb8904d98e0ee288704e6e16a0def95dbb850d4687'
params = {
    'fsym': 'ETH',               
    'tsym': 'BRL',              
    'limit': 2000,              
    'api_key': api  
}

response = requests.get(url, params=params)
dados = response.json()

df_ethereum = pd.DataFrame(dados['Data']['Data'])
df_ethereum = df_ethereum.drop(columns=['volumeto','conversionType', 'conversionSymbol'])
df_ethereum = df_ethereum[['time','open','high','low','close','volumefrom']]
df_ethereum.columns = ['Data','Abertura','Máxima','Mínima','Fechamento','Volume']
df_ethereum['Variação'] = df_ethereum['Fechamento'].pct_change().mul(100).round(2)
df_ethereum['Data'] = pd.to_datetime(df_ethereum['Data'], unit='s').dt.strftime('%d/%m/%Y')
df_ethereum = df_ethereum[::-1]

df_ethereum.head(5)

,Data,Abertura,Máxima,Mínima,Fechamento,Volume,Variação
2000,28/06/2025,13381.57,13537.96,13269.28,13462.87,340656.63,0.64
1999,27/06/2025,13324.14,13589.69,13258.66,13376.89,28220.36,0.40
1998,26/06/2025,13505.39,13754.91,13235.63,13324.14,45208.02,-1.34
1997,25/06/2025,13529.10,13877.04,13349.53,13505.39,52136.56,-0.18
1996,24/06/2025,13295.16,13699.01,13312.30,13529.10,65147.13,1.76


In [12]:
#https://www.alphavantage.co/documentation/#fx-daily
#Dolar x Real

url = 'https://www.alphavantage.co/query'
api = 'XUVYGH7HQ1CATKYS'
params = {
    'function': 'FX_DAILY',
    'from_symbol': 'USD',
    'to_symbol': 'BRL',
    'outputsize': 'full',
    'apikey': api
}

response = requests.get(url, params=params)
dados = response.json()

# Pega apenas a seção dos preços
precos = dados['Time Series FX (Daily)']

# Transforma em DataFrame
df_dolar = pd.DataFrame.from_dict(precos, orient='index')
df_dolar.reset_index(inplace=True)
df_dolar.columns = ['Data','Abertura', 'Máxima', 'Mínima', 'Fechamento']
colunas = ['Abertura', 'Máxima', 'Mínima','Fechamento']
df_dolar[colunas] = df_dolar[colunas].astype(float)

# Converte a data e ordena do mais antigo para o mais recente (necessário para pct_change)
df_dolar['Data'] = pd.to_datetime(df_dolar['Data'])
df_dolar = df_dolar.sort_values('Data').reset_index(drop=True)

# Calcula variação percentual
df_dolar['Variação'] = df_dolar['Fechamento'].pct_change().mul(100).round(2)

# Reordena para exibir da data mais recente para a mais antiga
df_dolar = df_dolar.sort_values('Data', ascending=False).reset_index(drop=True)
df_dolar['Data'] = df_dolar['Data'].dt.strftime('%d/%m/%Y')

df_dolar.head()


,Data,Abertura,Máxima,Mínima,Fechamento,Variação
0,27/06/2025,5.4932,5.5039,5.4577,5.4777,-0.02
1,26/06/2025,5.5490,5.5640,5.4763,5.4788,-1.27
2,25/06/2025,5.5176,5.5736,5.4997,5.5490,0.69
3,24/06/2025,5.5020,5.5230,5.4748,5.5109,0.30
4,23/06/2025,5.5129,5.5428,5.4931,5.4946,-0.33


In [10]:
url = 'https://www.alphavantage.co/query'
api = 'XUVYGH7HQ1CATKYS'
params = {
    'function': 'FX_DAILY',
    'from_symbol': 'EUR',
    'to_symbol': 'BRL',
    'outputsize': 'full',
    'apikey': api  # substitua pela sua chave
}

response = requests.get(url, params=params)
dados = response.json()



# Transforma em DataFrame
df_euro = pd.DataFrame.from_dict(precos, orient='index')
df_euro.reset_index(inplace=True)
df_euro.columns = ['Data', 'Abertura', 'Máxima', 'Mínima', 'Fechamento']
    
# Converte tipos
colunas = ['Abertura', 'Máxima', 'Mínima', 'Fechamento']
df_euro[colunas] = df_euro[colunas].astype(float)

# Converte a data e ordena cronologicamente
df_euro['Data'] = pd.to_datetime(df_euro['Data'])
df_euro = df_euro.sort_values('Data').reset_index(drop=True)

# Calcula a variação percentual
df_euro['Variação'] = df_euro['Fechamento'].pct_change().mul(100).round(2)

# Reordena para exibição (mais recente primeiro)
df_euro = df_euro.sort_values('Data', ascending=False).reset_index(drop=True)
df_euro['Data'] = df_euro['Data'].dt.strftime('%d/%m/%Y')
    
df_euro.head(5)

,Data,Abertura,Máxima,Mínima,Fechamento,Variação
0,27/06/2025,5.4932,5.5039,5.4577,5.4777,-0.02
1,26/06/2025,5.5490,5.5640,5.4763,5.4788,-1.27
2,25/06/2025,5.5176,5.5736,5.4997,5.5490,0.69
3,24/06/2025,5.5020,5.5230,5.4748,5.5109,0.30
4,23/06/2025,5.5129,5.5428,5.4931,5.4946,-0.33
